# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





Day23 東森新聞雲爬蟲練習

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import _thread
import time

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)

In [4]:
SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [10]:
def getNewsDetailContent(news_link):
    link_url="https://www.ettoday.net" + news_link.get('href')
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
#     for p in news_content:
#         if ((p.string) is not None):
#             print(p.string)

In [7]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_links = all_news.select('.piece.clearfix>a')

In [11]:
# 正常情況的爬蟲所需時間
startTime = time.time()

for news_link in news_links:
    getNewsDetailContent(news_link)

finishTime = time.time()
print(finishTime - startTime)

14.742146253585815


In [12]:
# 利用 thread 的爬蟲所需時間
startTime = time.time()

for news_link in news_links:
    _thread.start_new_thread( getNewsDetailContent, (news_link, ) ) 

finishTime = time.time()
print(finishTime - startTime)

0.01700425148010254


In [13]:
browser.close()